In [1]:
import os
import libtts

import numpy as np

Error @ Downsampling. C++ module 'libtts' not found. Using Python-only implementation.


In [ ]:
tls_veg_file = "close_stems_3.pts"

In [ ]:
### generate alpha shape
th_alpha_sq = 0.01
as_ret, as_file = libtts.alpha_shape_generation(tls_veg_file, th_alpha_sq)
print("Alpha shape generation result:", as_ret)
print("Alpha shape file:", as_file)

In [ ]:
### detect tree locations
treeloc_file = f"{tls_veg_file[:-4]}_treeloc.pts"
libtts.detect_trees(tls_veg_file, th_low_base = 0.5, th_low_length = 0.5, th_avg_knn_dist = 1.0, th_db_eps = 0.2, out_file=treeloc_file)

In [ ]:
### tree segmentation
print(f"{as_file=}, {treeloc_file=}")
segfile = f"{tls_veg_file[:-4]}_lbl.pts"
libtts.extract_single_trees(as_file, treeloc_file, segfile)

In [ ]:
### label remaining points
tls_veg_file = "close_stems_3.pts"
segfile = f"{tls_veg_file[:-4]}_a0.010_lbl.pts"

all_pts = np.loadtxt(tls_veg_file)
initial_labels = np.loadtxt(segfile)
all_lbl_file = f"{tls_veg_file[:-4]}_lbl_all.pts"
#ret = libtts.label_points_layered_nn(all_pts, initial_labels, layer_height=0.5, max_search_radius=2.0, out_file=all_lbl_file)
ret = libtts.label_points_region_growing(all_pts, initial_labels, search_radius=0.5, out_file=all_lbl_file)

Starting Z-Ordered Region Growing with radius: 0.5
Found 3 unique labels to grow from.
Z-Ordered Region Growing complete.
Labels saved to close_stems_3_lbl_all.pts


In [ ]:
# add function: label remaining points

In [ ]:
# ALS
als_infile = ""
libtts.als_segment(als_infile)